**Chapter 11: API Security**

## Introduction: The API Attack Surface

Application Programming Interfaces (APIs) have become the connective tissue of modern digital infrastructure. Whether you're building microservices, mobile applications, or integrating third-party services, APIs handle the majority of internet traffic today. However, this ubiquity makes them prime targets for attackers.

Unlike traditional web applications that primarily serve HTML to browsers, APIs expose direct programmatic access to backend systems. A single vulnerable endpoint can provide attackers with direct database access, administrative functions, or the ability to exfiltrate sensitive data at scale. According to recent industry reports, API attacks have increased by over 400% in the past three years, with broken authentication and authorization remaining the most prevalent vulnerabilities.

This chapter will guide you through securing APIs from the ground up, following the OWASP API Security Top 10 (2023) framework and industry standards including OAuth 2.1, OpenID Connect, and the OpenAPI Specification. By the end, you'll understand how to design, implement, and maintain APIs that can withstand real-world attacks.

---

## 11.1 API Security Fundamentals

### Understanding API Architectures and Their Security Implications

Before securing an API, you must understand its architectural style, as each presents unique security challenges.

#### Representational State Transfer (REST)

REST remains the dominant architectural style for web APIs. It uses standard HTTP methods (GET, POST, PUT, DELETE) to perform CRUD operations on resources identified by URLs.

**Security Characteristics:**
- **Statelessness**: Each request must contain all information needed for authentication/authorization, typically via headers
- **Resource-based**: Security must be enforced at the resource level (e.g., `/api/users/123` requires verifying the caller can access user 123)
- **Cacheability**: Sensitive data can be inadvertently cached by proxies if headers aren't properly set

**Example REST Endpoint Structure:**
```
GET    /api/v1/users/{id}      # Retrieve user
POST   /api/v1/users           # Create user
PUT    /api/v1/users/{id}      # Update user
DELETE /api/v1/users/{id}      # Delete user
```

#### GraphQL

GraphQL allows clients to query exactly the data they need through a single endpoint, but this flexibility introduces unique security concerns.

**Security Differences from REST:**
- **Query Complexity**: Clients can craft deeply nested queries that cause denial of service through resource exhaustion
- **Field-level Authorization**: Unlike REST where entire endpoints are protected, GraphQL requires checking permissions for individual fields
- **Introspection**: Production GraphQL APIs often expose schema details that aid attackers

**Example GraphQL Query:**
```graphql
query {
  user(id: "123") {
    name
    email
    orders {
      total
      items {
        price
      }
    }
  }
}
```

#### gRPC (Google Remote Procedure Call)

gRPC uses Protocol Buffers (protobuf) for efficient binary serialization over HTTP/2. It's common in microservices communication.

**Security Considerations:**
- **Binary Protocol**: Harder to inspect than JSON/XML without proper tooling
- **HTTP/2 Requirements**: Different load balancing and rate limiting strategies needed
- **mTLS**: gRPC strongly relies on mutual TLS for service-to-service authentication

### The API Security Triangle

Regardless of architecture, API security rests on three pillars:

1. **Confidentiality**: Ensuring data is encrypted in transit (TLS 1.3) and at rest
2. **Integrity**: Verifying data hasn't been tampered with (signatures, HMACs)
3. **Availability**: Preventing abuse through rate limiting and resource controls

---

## 11.2 Authentication & Authorization for APIs

Authentication (verifying identity) and Authorization (determining permissions) are distinct but complementary processes. Confusing these is a common source of vulnerabilities.

### API Keys: Simple but Risky

API keys are identifiers passed in headers to identify the calling application (not the user). They're suitable for server-to-server communication but should never be used for user authentication.

**Secure API Key Implementation:**

```python
import secrets
import hashlib
import base64
from datetime import datetime, timedelta

class APIKeyManager:
    def __init__(self):
        self.key_store = {}  # In production, use secure database/Vault
    
    def generate_key(self, client_id, permissions, expiry_days=90):
        """
        Generate a cryptographically secure API key
        Format: prefix.random_string.hash
        """
        # Generate 32 bytes of randomness
        random_component = secrets.token_urlsafe(32)
        
        # Create prefix for identification (e.g., 'live_' or 'test_')
        prefix = "ak_live_"
        
        # Combine and hash for storage (never store raw key)
        raw_key = f"{prefix}{random_component}"
        key_hash = hashlib.sha256(raw_key.encode()).hexdigest()
        
        # Store metadata
        self.key_store[key_hash] = {
            'client_id': client_id,
            'permissions': permissions,
            'created_at': datetime.utcnow(),
            'expires_at': datetime.utcnow() + timedelta(days=expiry_days),
            'last_used': None
        }
        
        return raw_key  # Return only once to client
    
    def validate_key(self, api_key):
        """Validate key without timing side-channels"""
        if not api_key or not api_key.startswith("ak_live_"):
            return False
            
        provided_hash = hashlib.sha256(api_key.encode()).hexdigest()
        
        # Constant-time comparison to prevent timing attacks
        expected_hash = self.key_store.get(provided_hash)
        if not expected_hash:
            return False
            
        # Check expiration
        if datetime.utcnow() > expected_hash['expires_at']:
            return False
            
        return True
```

**Critical API Key Security Practices:**
- Never embed keys in client-side code (JavaScript, mobile apps)
- Rotate keys every 90 days or after compromise
- Use different keys for different environments (dev/staging/prod)
- Implement key scoping (read-only vs. read-write)
- Monitor for anomalous usage patterns

### OAuth 2.0 and OpenID Connect (OIDC)

For user authentication and delegated authorization, OAuth 2.0 is the industry standard. OIDC adds an identity layer on top of OAuth 2.0.

#### Understanding OAuth 2.0 Flows

**1. Authorization Code Flow (with PKCE)**
The gold standard for single-page applications (SPAs) and mobile apps. PKCE (Proof Key for Code Exchange) prevents authorization code interception attacks.

```javascript
// Client-side: Initiating OAuth 2.0 with PKCE (React example)
import { generateCodeChallenge, generateCodeVerifier } from './pkce-utils';

async function initiateOAuth() {
    // Generate cryptographically random verifier
    const codeVerifier = generateCodeVerifier();
    const codeChallenge = await generateCodeChallenge(codeVerifier);
    
    // Store verifier for later (sessionStorage is safer than localStorage)
    sessionStorage.setItem('pkce_verifier', codeVerifier);
    
    const params = new URLSearchParams({
        response_type: 'code',
        client_id: 'your-client-id',
        redirect_uri: 'https://yourapp.com/callback',
        scope: 'openid profile api:read',
        code_challenge: codeChallenge,
        code_challenge_method: 'S256', // SHA256
        state: generateRandomState() // Prevent CSRF
    });
    
    window.location.href = `https://auth-server.com/oauth/authorize?${params}`;
}

// Server-side: Exchanging code for tokens (Node.js/Express)
const express = require('express');
const axios = require('axios');
const app = express();

app.post('/oauth/callback', async (req, res) => {
    const { code, state } = req.body;
    const codeVerifier = req.session.pkce_verifier; // Retrieved from secure session
    
    // Verify state parameter matches (CSRF protection)
    if (state !== req.session.oauth_state) {
        return res.status(403).json({ error: 'Invalid state parameter' });
    }
    
    try {
        const tokenResponse = await axios.post('https://auth-server.com/oauth/token', {
            grant_type: 'authorization_code',
            code: code,
            redirect_uri: 'https://yourapp.com/callback',
            client_id: process.env.CLIENT_ID,
            client_secret: process.env.CLIENT_SECRET, // Server-side only
            code_verifier: codeVerifier
        });
        
        const { access_token, id_token, refresh_token } = tokenResponse.data;
        
        // Validate tokens (see JWT section below)
        // Store securely (httpOnly cookies preferred over localStorage)
        res.cookie('access_token', access_token, { 
            httpOnly: true, 
            secure: true, 
            sameSite: 'strict',
            maxAge: 3600000 // 1 hour
        });
        
        res.json({ success: true });
    } catch (error) {
        res.status(400).json({ error: 'Token exchange failed' });
    }
});
```

**2. Client Credentials Flow**
For machine-to-machine communication where no user is involved.

```python
import requests
from requests.auth import HTTPBasicAuth

def get_machine_token():
    """
    Client Credentials Flow for service-to-service auth
    """
    response = requests.post(
        'https://auth-server.com/oauth/token',
        auth=HTTPBasicAuth('client_id', 'client_secret'),
        data={
            'grant_type': 'client_credentials',
            'scope': 'api:internal'
        },
        headers={'Content-Type': 'application/x-www-form-urlencoded'}
    )
    
    if response.status_code == 200:
        token_data = response.json()
        return token_data['access_token']
    else:
        raise Exception(f"Authentication failed: {response.text}")
```

**3. Device Authorization Flow**
For devices with limited input capabilities (smart TVs, IoT devices).

#### JWT (JSON Web Tokens) Security

JWTs are the standard format for access tokens. Understanding their structure is crucial for secure implementation.

**JWT Structure:**
```
header.payload.signature
```

**Example JWT Claims:**
```json
// Header
{
  "alg": "RS256",  // Algorithm: RSA with SHA-256 (preferred over HS256)
  "typ": "JWT",
  "kid": "key-id-1"  // Key ID for rotating keys
}

// Payload
{
  "sub": "user123",        // Subject
  "iss": "https://auth.company.com", // Issuer
  "aud": "api.company.com", // Audience (who can use this token)
  "exp": 1735689600,       // Expiration (Unix timestamp)
  "iat": 1735686000,       // Issued at
  "scope": "read:users write:orders",
  "jti": "unique-token-id" // JWT ID for revocation lists
}
```

**Secure JWT Validation (Python with PyJWT):**

```python
import jwt
from jwt.exceptions import InvalidTokenError, ExpiredSignatureError
import requests
from cryptography.hazmat.primitives import serialization

class JWTValidator:
    def __init__(self, jwks_url, expected_issuer, expected_audience):
        """
        Initialize with JWKS endpoint for key rotation support
        """
        self.jwks_url = jwks_url
        self.expected_issuer = expected_issuer
        self.expected_audience = expected_audience
        self.jwks_client = jwt.PyJWKClient(jwks_url)
    
    def validate_token(self, token):
        """
        Comprehensive JWT validation following RFC 8725 (JWT Best Practices)
        """
        try:
            # Get signing key from JWKS
            signing_key = self.jwks_client.get_signing_key_from_jwt(token)
            
            # Decode and validate all critical claims
            payload = jwt.decode(
                token,
                signing_key.key,
                algorithms=['RS256', 'ES256'], # Explicitly allow only asymmetric algorithms
                issuer=self.expected_issuer,
                audience=self.expected_audience,
                options={
                    "verify_exp": True,
                    "verify_iat": True,
                    "verify_nbf": True,
                    "require": ["exp", "iat", "sub", "iss", "aud"]
                }
            )
            
            # Additional custom validation
            if payload.get('jti') in self.revocation_list:
                raise InvalidTokenError("Token has been revoked")
            
            return payload
            
        except ExpiredSignatureError:
            raise InvalidTokenError("Token has expired")
        except InvalidTokenError as e:
            raise InvalidTokenError(f"Invalid token: {str(e)}")

# Usage in Flask middleware
from flask import request, jsonify

@app.before_request
def check_jwt():
    if request.endpoint == 'public':
        return
    
    auth_header = request.headers.get('Authorization')
    if not auth_header or not auth_header.startswith('Bearer '):
        return jsonify({"error": "Missing or invalid Authorization header"}), 401
    
    token = auth_header.split(' ')[1]
    validator = JWTValidator(
        jwks_url="https://auth.company.com/.well-known/jwks.json",
        expected_issuer="https://auth.company.com",
        expected_audience="api.company.com"
    )
    
    try:
        decoded = validator.validate_token(token)
        request.current_user = decoded['sub']
        request.user_scopes = decoded.get('scope', '').split()
    except InvalidTokenError as e:
        return jsonify({"error": str(e)}), 401
```

**Critical JWT Security Rules:**
1. **Never use `none` algorithm** - Always explicitly specify allowed algorithms
2. **Validate all claims**: `exp`, `iss`, `aud`, `sub`
3. **Use asymmetric algorithms** (RS256, ES256) for distributed validation; symmetric (HS256) only when the validator knows the secret
4. **Short expiration**: Access tokens should expire in 15-60 minutes
5. **Secure storage**: Never store JWTs in localStorage (XSS vulnerable); use httpOnly cookies or secure memory
6. **Key rotation**: Implement JWKS (JSON Web Key Set) endpoints for seamless key rotation

---

## 11.3 Common API Vulnerabilities (OWASP API Security Top 10 2023)

The OWASP API Security Top 10 identifies the most critical API vulnerabilities. Understanding these is essential for defensive coding.

### API1:2023 – Broken Object Level Authorization (BOLA)

Previously known as Insecure Direct Object Reference (IDOR), BOLA occurs when an API exposes objects (database records, files) without verifying the user owns them.

**Vulnerable Code (Python/Flask):**
```python
@app.route('/api/orders/<order_id>')
@login_required
def get_order(order_id):
    # DANGER: No check if current user owns this order
    order = db.execute(f"SELECT * FROM orders WHERE id = {order_id}")
    return jsonify(order)
```

**Attack Scenario:**
User A is authenticated. User A accesses `/api/orders/10025` (their order) successfully. They then try `/api/orders/10026` and receive User B's order details because the API doesn't verify ownership.

**Secure Implementation:**
```python
@app.route('/api/orders/<order_id>')
@login_required
def get_order(order_id):
    user_id = get_current_user_id()
    
    # Secure: Check ownership in query
    order = db.execute(
        "SELECT * FROM orders WHERE id = ? AND user_id = ?", 
        (order_id, user_id)
    ).fetchone()
    
    if not order:
        # Return 404 (not 403) to prevent user enumeration
        abort(404, description="Order not found")
    
    return jsonify(order)
```

**BOLA Prevention Checklist:**
- Implement authorization checks at the data layer, not just the API layer
- Use random, unpredictable IDs (UUIDs) instead of sequential integers
- Prefer indirect reference maps (IRM) for sensitive resources
- Test with automated tools that attempt horizontal privilege escalation

### API2:2023 – Broken Authentication

Authentication mechanisms are often implemented incorrectly, allowing attackers to compromise tokens, passwords, or session identifiers.

**Common Failures:**
- Weak password policies and brute-force protection
- JWT secrets hardcoded or weak
- Session tokens with insufficient entropy
- Missing multi-factor authentication (MFA)

**Implementing Secure Rate Limiting:**
```python
from flask_limiter import Limiter
from flask_limiter.util import get_remote_address

limiter = Limiter(
    app=app,
    key_func=get_remote_address,
    default_limits=["200 per day", "50 per hour"]
)

@app.route('/api/login', methods=['POST'])
@limiter.limit("5 per minute")  # Stricter limits for auth endpoints
def login():
    username = request.json.get('username')
    password = request.json.get('password')
    
    # Constant-time comparison to prevent timing attacks
    if not verify_password_constant_time(username, password):
        # Generic error message to prevent user enumeration
        return jsonify({"error": "Invalid credentials"}), 401
    
    # Generate session or token...
    return jsonify({"token": generate_token(username)})

@app.errorhandler(429)
def ratelimit_handler(e):
    return jsonify({
        "error": "Rate limit exceeded",
        "retry_after": e.description
    }), 429
```

### API3:2023 – Broken Object Property Level Authorization (BOPLA)

A new addition to the 2023 list, BOPLA combines Mass Assignment and Excessive Data Exposure. It occurs when APIs expose more object properties than intended or allow clients to modify properties they shouldn't.

**Vulnerable Code (Node.js/Express):**
```javascript
// PUT /api/users/123
app.put('/api/users/:id', authenticate, (req, res) => {
    // DANGER: Directly passing req.body allows modifying any field
    User.findByIdAndUpdate(req.params.id, req.body, (err, user) => {
        res.json(user);
    });
});

// Attack: PUT /api/users/123 with body {"role": "admin", "password": "newpass"}
```

**Secure Implementation with Allowlist:**
```javascript
const ALLOWED_FIELDS = ['name', 'email', 'phone', 'address'];

app.put('/api/users/:id', authenticate, async (req, res) => {
    try {
        // Verify ownership
        if (req.user.id !== req.params.id && req.user.role !== 'admin') {
            return res.status(403).json({ error: 'Unauthorized' });
        }
        
        // Filter allowed fields only
        const updates = {};
        ALLOWED_FIELDS.forEach(field => {
            if (req.body[field] !== undefined) {
                updates[field] = req.body[field];
            }
        });
        
        // Prevent privilege escalation
        delete updates.role;
        delete updates.id;
        delete updates.createdAt;
        
        const user = await User.findByIdAndUpdate(
            req.params.id, 
            updates, 
            { new: true, runValidators: true }
        );
        
        res.json(user);
    } catch (error) {
        res.status(400).json({ error: error.message });
    }
});
```

### API4:2023 – Unrestricted Resource Consumption

APIs without limits on resource usage are vulnerable to Denial of Service (DoS) and financial exhaustion (if using pay-per-use cloud resources).

**GraphQL Query Complexity Attack:**
```graphql
# This query could exponentially join data and crash the server
query malicious {
  users {
    orders {
      items {
        product {
          reviews {
            user {
              orders {
                # ... infinite nesting possible
              }
            }
          }
        }
      }
    }
  }
}
```

**GraphQL Depth Limiting (Node.js):**
```javascript
const { createComplexityLimitRule } = require('graphql-validation-complexity');
const depthLimit = require('graphql-depth-limit');

const server = new ApolloServer({
    typeDefs,
    resolvers,
    validationRules: [
        depthLimit(5), // Maximum nesting depth
        createComplexityLimitRule(1000, {
            onComplete: (complexity) => {
                console.log('Query complexity:', complexity);
            }
        })
    ]
});
```

**Pagination and Limits:**
```python
@app.route('/api/users')
@login_required
def get_users():
    # Enforce maximum limit
    limit = min(int(request.args.get('limit', 20)), 100)
    offset = int(request.args.get('offset', 0))
    
    # Use cursor-based pagination for large datasets (better performance)
    cursor = request.args.get('cursor')
    
    if cursor:
        users = User.query.filter(User.id > cursor).limit(limit).all()
    else:
        users = User.query.limit(limit).offset(offset).all()
    
    return jsonify({
        'data': users,
        'pagination': {
            'limit': limit,
            'next_cursor': users[-1].id if users else None
        }
    })
```

### API5:2023 – Broken Function Level Authorization

Occurs when administrative or sensitive functions lack proper authorization checks, often through "Security through Obscurity" (hiding admin endpoints instead of protecting them).

**Vulnerable Pattern:**
```javascript
// Admin function supposedly hidden
app.delete('/api/admin/users/:id', (req, res) => {
    // Missing admin role check!
    User.findByIdAndDelete(req.params.id);
    res.json({ message: 'User deleted' });
});
```

**Defense in Depth:**
```python
from functools import wraps
from flask import abort

def require_role(allowed_roles):
    def decorator(f):
        @wraps(f)
        def decorated_function(*args, **kwargs):
            token = extract_token_from_header()
            payload = jwt.decode(token, verify=True)
            
            user_role = payload.get('role')
            if user_role not in allowed_roles:
                abort(403, description="Insufficient privileges")
            
            return f(*args, **kwargs)
        return decorated_function
    return decorator

@app.route('/api/admin/users/<user_id>', methods=['DELETE'])
@require_role(['admin', 'superadmin'])
def delete_user(user_id):
    # Additional ownership/scope checks...
    pass
```

### API6:2023 – Unrestricted Access to Sensitive Business Flows

Automated abuse of business logic (scraping, mass account creation, ticket scalping) without technical vulnerabilities.

**Protection Strategies:**
- **Device fingerprinting**: Track browser characteristics, canvas fingerprints
- **Behavioral analysis**: Detect non-human patterns (perfect rhythm, no mouse movements)
- **Proof of Work**: Require computational challenges for sensitive operations
- **CAPTCHA**: Use invisible reCAPTCHA v3 for suspicious traffic

### API7:2023 – Server Side Request Forgery (SSRF)

Occurs when an API fetches remote resources without validating the URL, allowing attackers to force the server to make requests to internal services.

**Vulnerable Code:**
```python
@app.route('/api/fetch-metadata')
def fetch_metadata():
    url = request.args.get('url')
    # DANGER: No validation of URL
    response = requests.get(url)
    return response.text

# Attack: /api/fetch-metadata?url=http://localhost:6379/ 
# (Accessing internal Redis instance)
```

**Secure Implementation with Allowlist:**
```python
import urllib.parse
import ipaddress

def is_safe_url(url):
    """
    Validate URL to prevent SSRF
    """
    parsed = urllib.parse.urlparse(url)
    
    # Allowlist protocols
    if parsed.scheme not in ['http', 'https']:
        return False
    
    # Resolve hostname
    try:
        hostname = parsed.hostname
        if not hostname:
            return False
            
        # Check for IP addresses
        try:
            ip = ipaddress.ip_address(hostname)
            # Block private/reserved IPs
            if ip.is_private or ip.is_reserved or ip.is_loopback:
                return False
        except ValueError:
            # It's a domain name, check against allowlist
            allowed_domains = ['api.trusted-source.com', 'cdn.example.com']
            if not any(hostname.endswith(domain) for domain in allowed_domains):
                return False
                
        return True
    except Exception:
        return False

@app.route('/api/fetch-metadata')
def fetch_metadata():
    url = request.args.get('url')
    
    if not is_safe_url(url):
        return jsonify({"error": "Invalid URL"}), 400
    
    # Use timeouts and limit redirects
    response = requests.get(
        url, 
        timeout=5,
        allow_redirects=False,
        headers={'User-Agent': 'API-Validator/1.0'}
    )
    
    return response.text
```

### API8:2023 – Security Misconfiguration

Includes unnecessary features enabled, default accounts/passwords, error messages with sensitive info, and missing security headers.

**Security Headers for APIs:**
```python
@app.after_request
def add_security_headers(response):
    # Prevent MIME type sniffing
    response.headers['X-Content-Type-Options'] = 'nosniff'
    
    # Prevent clickjacking
    response.headers['X-Frame-Options'] = 'DENY'
    
    # Strict Transport Security (force HTTPS)
    response.headers['Strict-Transport-Security'] = 'max-age=31536000; includeSubDomains'
    
    # Content Security Policy (for API documentation pages)
    response.headers['Content-Security-Policy'] = "default-src 'self'"
    
    # Cache control for sensitive data
    response.headers['Cache-Control'] = 'no-store, no-cache, must-revalidate, private'
    response.headers['Pragma'] = 'no-cache'
    
    return response
```

### API9:2023 – Improper Inventory Management

Shadow APIs (unknown/undocumented), deprecated API versions, and debug endpoints in production.

**API Inventory Best Practices:**
- Maintain an OpenAPI specification for all endpoints
- Implement API versioning strategy (URL or header-based)
- Use API discovery tools to find shadow endpoints
- Sunset old versions with proper deprecation warnings

**API Versioning Example:**
```
/api/v1/users      # Current stable
/api/v2/users      # New version with breaking changes
/api/beta/users    # Experimental (restricted access)
```

### API10:2023 – Unsafe Consumption of APIs

When your API calls third-party APIs without proper validation of responses, or transmits sensitive data to them.

**Secure Third-Party Integration:**
```python
import requests
from requests.adapters import HTTPAdapter
from urllib3.util.retry import Retry

def safe_third_party_call():
    """
    Securely call external APIs with timeouts, retries, and validation
    """
    session = requests.Session()
    
    # Implement retry strategy with backoff
    retry_strategy = Retry(
        total=3,
        backoff_factor=1,
        status_forcelist=[429, 500, 502, 503, 504],
    )
    
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("https://", adapter)
    
    try:
        response = session.get(
            'https://third-party-api.com/data',
            timeout=(3, 27),  # (connect timeout, read timeout)
            headers={'Accept': 'application/json'}
        )
        response.raise_for_status()
        
        # Validate response structure before processing
        data = response.json()
        if not validate_schema(data, EXPECTED_SCHEMA):
            raise ValueError("Unexpected response structure")
            
        return data
        
    except requests.exceptions.Timeout:
        # Log but don't expose internal details to client
        logger.error("Third party API timeout")
        raise ServiceUnavailable()
    except requests.exceptions.RequestException as e:
        logger.error(f"Third party API error: {e}")
        raise ServiceUnavailable()
```

---

## 11.4 API Gateway Security

An API Gateway acts as a reverse proxy, centralizing cross-cutting concerns like authentication, rate limiting, and logging.

### Rate Limiting Strategies

**Token Bucket Algorithm:**
Allows bursts up to a maximum capacity while maintaining average rate.

```python
import redis
import time

class TokenBucket:
    def __init__(self, redis_client, key, capacity, refill_rate):
        self.redis = redis_client
        self.key = key
        self.capacity = capacity
        self.refill_rate = refill_rate
    
    def is_allowed(self):
        """
        Check if request is allowed under rate limit
        Returns: (allowed: bool, remaining: int, reset_time: int)
        """
        pipe = self.redis.pipeline()
        now = time.time()
        
        # Lua script for atomic operation
        lua_script = """
        local key = KEYS[1]
        local capacity = tonumber(ARGV[1])
        local refill_rate = tonumber(ARGV[2])
        local now = tonumber(ARGV[3])
        
        local bucket = redis.call('HMGET', key, 'tokens', 'last_refill')
        local tokens = tonumber(bucket[1]) or capacity
        local last_refill = tonumber(bucket[2]) or now
        
        -- Calculate tokens to add
        local delta = now - last_refill
        local new_tokens = math.min(capacity, tokens + (delta * refill_rate))
        
        if new_tokens >= 1 then
            new_tokens = new_tokens - 1
            redis.call('HMSET', key, 'tokens', new_tokens, 'last_refill', now)
            redis.call('EXPIRE', key, 3600)
            return {1, math.floor(new_tokens), math.floor(now + (1/refill_rate))}
        else
            redis.call('HMSET', key, 'tokens', new_tokens, 'last_refill', now)
            redis.call('EXPIRE', key, 3600)
            local retry_after = math.ceil((1 - new_tokens) / refill_rate)
            return {0, 0, retry_after}
        end
        """
        
        result = self.redis.eval(
            lua_script, 1, self.key, 
            self.capacity, self.refill_rate, now
        )
        
        allowed = bool(result[0])
        remaining = result[1]
        reset = result[2]
        
        return allowed, remaining, reset

# Flask middleware integration
@app.before_request
def rate_limit():
    client_id = get_client_id()  # From API key or IP
    bucket = TokenBucket(redis_client, f"rate_limit:{client_id}", 100, 10)  # 100 burst, 10/sec
    
    allowed, remaining, reset = bucket.is_allowed()
    
    # Add rate limit headers (RFC 6585)
    response_headers = {
        'X-RateLimit-Limit': '100',
        'X-RateLimit-Remaining': str(remaining),
        'X-RateLimit-Reset': str(reset)
    }
    
    if not allowed:
        response = jsonify({"error": "Rate limit exceeded"})
        response.status_code = 429
        response.headers.update(response_headers)
        response.headers['Retry-After'] = str(reset - int(time.time()))
        return response
    
    # Store headers to be added to actual response
    g.rate_limit_headers = response_headers

@app.after_request
def add_rate_headers(response):
    if hasattr(g, 'rate_limit_headers'):
        response.headers.update(g.rate_limit_headers)
    return response
```

### Input Validation at the Edge

Validate input as early as possible (at the gateway) to reduce load on backend services.

**JSON Schema Validation:**
```yaml
# OpenAPI Specification with validation rules
paths:
  /api/users:
    post:
      requestBody:
        required: true
        content:
          application/json:
            schema:
              type: object
              required: [email, name]
              properties:
                email:
                  type: string
                  format: email
                  maxLength: 254
                name:
                  type: string
                  minLength: 2
                  maxLength: 100
                  pattern: '^[a-zA-Z0-9\s-]+$'
                age:
                  type: integer
                  minimum: 13
                  maximum: 120
```

### Mutual TLS (mTLS) for Service-to-Service

For high-security environments, require client certificates:

```nginx
# NGINX configuration for mTLS
server {
    listen 443 ssl;
    server_name api.internal.company.com;
    
    ssl_certificate /etc/nginx/certs/server.crt;
    ssl_certificate_key /etc/nginx/certs/server.key;
    ssl_client_certificate /etc/nginx/certs/ca.crt;
    ssl_verify_client on;
    ssl_verify_depth 2;
    
    location / {
        # Pass client certificate info to backend
        proxy_set_header X-SSL-Client-S-DN $ssl_client_s_dn;
        proxy_set_header X-SSL-Client-I-DN $ssl_client_i_dn;
        proxy_set_header X-SSL-Client-Serial $ssl_client_serial;
        proxy_set_header X-SSL-Client-Verify $ssl_client_verify;
        
        proxy_pass http://backend;
    }
}
```

---

## 11.5 Secure API Design and Documentation

### OpenAPI Specification (OAS) Security

Documenting security schemes in your OpenAPI specification ensures consistent implementation and helps security tools scan your API.

```yaml
openapi: 3.0.3
info:
  title: Secure API Example
  version: 1.0.0

# Security schemes definition
components:
  securitySchemes:
    bearerAuth:
      type: http
      scheme: bearer
      bearerFormat: JWT
      description: JWT token obtained from /auth endpoint
    
    apiKeyAuth:
      type: apiKey
      in: header
      name: X-API-Key
    
    oauth2Auth:
      type: oauth2
      flows:
        authorizationCode:
          authorizationUrl: https://auth.company.com/oauth/authorize
          tokenUrl: https://auth.company.com/oauth/token
          scopes:
            read:users: Read user information
            write:users: Modify user information
            admin: Administrative access

security:
  - bearerAuth: []  # Default security for all endpoints

paths:
  /api/users:
    get:
      summary: List users
      security:
        - bearerAuth: [read:users]
        - apiKeyAuth: []  # Alternative auth method
      parameters:
        - name: limit
          in: query
          schema:
            type: integer
            minimum: 1
            maximum: 100
          description: Number of results to return
      responses:
        '200':
          description: Success
          content:
            application/json:
              schema:
                type: array
                items:
                  $ref: '#/components/schemas/User'
        '401':
          description: Unauthorized
        '403':
          description: Forbidden - insufficient scope
    
    post:
      summary: Create user
      security:
        - bearerAuth: [write:users]
      requestBody:
        required: true
        content:
          application/json:
            schema:
              $ref: '#/components/schemas/UserInput'
      responses:
        '201':
          description: Created

components:
  schemas:
    User:
      type: object
      properties:
        id:
          type: string
          format: uuid
          readOnly: true
        email:
          type: string
          format: email
        name:
          type: string
        role:
          type: string
          enum: [user, admin]
          readOnly: true  # Cannot be set directly via API
```

### Error Handling That Doesn't Leak Information

**Dangerous Error (Information Leakage):**
```json
{
  "error": "SQL Syntax error near 'users' WHERE id = 123",
  "stack_trace": "Traceback (most recent call last): File '/app/routes.py', line 45...",
  "query": "SELECT * FROM users WHERE id = 123 AND tenant_id = 456"
}
```

**Secure Error Response:**
```json
{
  "error": {
    "code": "RESOURCE_NOT_FOUND",
    "message": "The requested resource could not be found",
    "request_id": "req_7f8d9a2b3c4e",
    "documentation_url": "https://api.docs.company.com/errors/RESOURCE_NOT_FOUND"
  }
}
```

**Implementation:**
```python
import logging
import uuid
from flask import jsonify, request

logger = logging.getLogger('api.errors')

class APIError(Exception):
    def __init__(self, message, status_code=500, error_code="INTERNAL_ERROR", details=None):
        super().__init__(message)
        self.status_code = status_code
        self.error_code = error_code
        self.details = details or {}

@app.errorhandler(Exception)
def handle_error(error):
    request_id = getattr(request, 'request_id', str(uuid.uuid4()))
    
    if isinstance(error, APIError):
        # Known error, safe to return details
        response = {
            "error": {
                "code": error.error_code,
                "message": str(error),
                "request_id": request_id
            }
        }
        status_code = error.status_code
    else:
        # Unexpected error, log details but return generic message
        logger.error(f"Unexpected error [Request ID: {request_id}]", 
                    exc_info=True, 
                    extra={'request_id': request_id, 'path': request.path})
        
        response = {
            "error": {
                "code": "INTERNAL_ERROR",
                "message": "An unexpected error occurred",
                "request_id": request_id
            }
        }
        status_code = 500
    
    return jsonify(response), status_code

# Usage in routes
@app.route('/api/users/<user_id>')
def get_user(user_id):
    user = User.query.get(user_id)
    if not user:
        raise APIError(
            "User not found", 
            status_code=404, 
            error_code="USER_NOT_FOUND"
        )
    return jsonify(user.to_dict())
```

### Versioning and Deprecation Strategy

**URL Path Versioning (Recommended):**
```
/api/v1/users
/api/v2/users
```

**Header Versioning (Alternative):**
```
Accept: application/vnd.api+json;version=1
```

**Deprecation Headers:**
```python
@app.route('/api/v1/legacy-endpoint')
def legacy_endpoint():
    response = jsonify(data)
    
    # Signal deprecation
    response.headers['Deprecation'] = 'true'
    response.headers['Sunset'] = 'Sat, 31 Dec 2024 23:59:59 GMT'
    response.headers['Link'] = '</api/v2/new-endpoint>; rel="successor-version"'
    
    return response
```

---

## Summary and Transition

In this chapter, you learned that API security requires a defense-in-depth strategy spanning architecture, authentication, authorization, and infrastructure. Key takeaways include:

- **Authentication** establishes identity through secure methods like OAuth 2.0 with PKCE and short-lived JWTs, while **Authorization** enforces permissions at both the object and function levels (preventing BOLA and BOPLA).
- The **OWASP API Security Top 10 2023** provides a roadmap of critical vulnerabilities, from Broken Object Level Authorization to Unsafe Consumption of APIs, each requiring specific coding patterns and validation strategies.
- **Rate limiting, input validation, and mTLS** at the API Gateway layer provide essential perimeter defense, but backend services must always validate independently (never trust the client).
- **Secure design** documented through OpenAPI specifications with strict schemas ensures that security is built into the API contract from the start, not bolted on later.

However, even the most securely designed API can be compromised if the underlying code contains vulnerabilities like injection flaws, insecure deserialization, or memory safety issues. While APIs present a specific attack surface, they are ultimately implemented using the same coding languages and patterns as any other software.

In **Chapter 12: Secure Coding Practices & Patterns**, we will shift from architectural and protocol-level security down to the code itself. You will learn language-agnostic secure coding principles including input validation and output encoding, memory management techniques to prevent buffer overflows, secure error handling patterns, and how to conduct effective security code reviews. We will explore Static Application Security Testing (SAST) tools and build a secure coding checklist that you can apply to any programming language or framework, ensuring that your implementation matches the security of your API design.